In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
import coin
import yfinance as yf

In [ ]:
df = yf.download('IVVB11.SA BOVA11.SA', interval='15m', period='60d')['Adj Close']

In [ ]:
df.dropna(inplace=True)

In [ ]:
retorno, df_ln = coin.calc_ret_ln(df)

In [ ]:
retorno

In [ ]:
periodo = 1000
base = retorno

# Regressão

## Precos as is

In [ ]:
y = np.array(base['BOVA11.SA'].iloc[:periodo], dtype = float)
x = np.array(base.iloc[:periodo,1:], dtype = float)

In [ ]:
coef = coin.reg_m(x,y).params

In [3]:
import pandas as pd
dados = pd.read_csv('testes_coin.csv')

C:\Users\maike\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
dados.dropna(inplace=True)
dados

,Unnamed: 0,Ativo 1,Ativo 2,Inicio,Periodo,ADF Test
0,0,ABEV3.SA,AZUL4.SA,0.0,60.0,-21.488518
1,1,ABEV3.SA,AZUL4.SA,0.0,100.0,-21.874476
2,2,ABEV3.SA,AZUL4.SA,0.0,140.0,-21.877322
3,3,ABEV3.SA,AZUL4.SA,0.0,180.0,-21.890945
4,4,ABEV3.SA,AZUL4.SA,0.0,220.0,-21.838615
...,...,...,...,...,...,...
205015,205015,YDUQ3.SA,WEGE3.SA,400.0,60.0,-12.698535
205016,205016,YDUQ3.SA,WEGE3.SA,400.0,100.0,-12.698535
205017,205017,YDUQ3.SA,WEGE3.SA,400.0,140.0,-12.698535
205018,205018,YDUQ3.SA,WEGE3.SA,400.0,180.0,-12.698535


In [ ]:
import plotly.express as px

fig = plt.figure(figsize=(30,30))
fig = px.scatter_3d(base, x='IVVB11.SA', y='BRL=X', z='BOVA11.SA')
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

# Resíduos

In [ ]:
residuos = pd.DataFrame(columns=['residuos'])
residuos['residuos'] = coin.reg_m(x,y).resid

In [ ]:
coef

In [ ]:
#erro_planilha = base['BOVA11.SA'] - (base['IVVB11.SA']*coef[2] + base['BRL=X']*coef[1] + coef[0])
erro_planilha = base['BOVA11.SA'] - (base['IVVB11.SA']*coef[1] + coef[0])
resi = erro_planilha
#resi = residuos['residuos']

In [ ]:
coin.coint_model(resi)['ADF']

In [ ]:
residuos.plot(figsize=(20,7))
plt.axhline(y=resi.std()*2, linewidth = 2, color='red', linestyle = 'dashed')
plt.axhline(y=-resi.std()*2, linewidth = 2, color='red', linestyle = 'dashed')
plt.axhline(y=resi.std(), linewidth = 1, color='black', linestyle = 'dashed')
plt.axhline(y=-resi.std(), linewidth = 1, color='black', linestyle = 'dashed')
plt.axhline(y=0, linewidth = 1, color='black')

# Meia Vida

In [ ]:
half_life, ar_res, delta, lagged = coin.half_life(residuos.residuos)
print('halflife: ', half_life)

In [ ]:
coin.get_scatter_plot(delta, lagged, ar_res)

## cointegrado backtest

In [ ]:
difu = dict
cointegrado = 0
amostras = 0
for inicio in range(0, 1500, 50):
    for periodo in np.arange(50,310, 10):
        y = np.array(base['BOVA11.SA'].iloc[inicio:(inicio+periodo)], dtype = float)
        x = np.array(base.iloc[inicio:(inicio+periodo),1:], dtype = float)

        residuos = pd.DataFrame(columns=['residuos'])
        residuos['residuos'] = coin.reg_m(x,y).resid
        
        test = coin.coint_model(residuos)['ADF'][0]
        difu = {'1%': -3.4753253063120644,'5%': -2.881274703275226,'10%': -2.5772928360116873}

        amostras+=1

        if difu['1%'] >= test:
            cointegrado +=1

print(cointegrado, amostras)

In [6]:
pip install basicbacktester

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement basicbacktester
ERROR: No matching distribution found for basicbacktester


# Preditor

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 200)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

In [ ]:
y_pred = regressor.predict(X_test)
print(y_pred)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

In [ ]:
def resid(self):
    """The residuals of the model."""
    return self.model.endog - self.model.predict(
        self.params, self.model.exog)

# Acoes

In [ ]:
import acoes

In [ ]:
tickers = 'IVVB11.SA BOVA11.SA'
interval='15m'
period='60d'
retorno, _ = acoes.download_dados(tickers, interval, period)
base = retorno

step = 50
coin_min = 60
coin_max = 200
coin_step = 40

acoes.teste_coint(base, step, coin_min, coin_max, coin_step)

In [ ]:
import yfinance as yf
import coin

tickers = 'IVVB11.SA BOVA11.SA'
interval='15m'
period='60d'

df = yf.download(tickers, interval=interval, period=period)['Adj Close']
retorno, df_ln = coin.calc_ret_ln(df)


In [ ]:
retorno